#### Load Libraries and Demo Codes

In [1]:
import julia; julia.install(quiet=True)
from julia import Main

import numpy as np
import sympy as sp

import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

import holoviews as hv
from holoviews import opts
import panel as pn
import param

pn.extension('katex', 'plotly')
hv.extension('bokeh')

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [2]:
%%julia
using Pkg; Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, RowEchelon, Printf, Latexify, LaTeXStrings, Random, SymPy

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


In [3]:
#%output backend="plotly"
def spherical_to_cartesian(r, theta_deg, phi_deg):
    theta = np.radians(theta_deg)
    phi = np.radians(phi_deg)
    return r * np.array([
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta)
    ])
def rotation_matrix(axis, theta):
    """
    Returns the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.linalg.norm(axis)
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    return np.array([
        [a*a + b*b - c*c - d*d, 2*(b*c - a*d),     2*(b*d + a*c)],
        [2*(b*c + a*d),     a*a + c*c - b*b - d*d, 2*(c*d - a*b)],
        [2*(b*d - a*c),     2*(c*d + a*b),     a*a + d*d - b*b - c*c]
    ])
def project_to_plane(v, normal):
    n = normal / np.linalg.norm(normal)
    return v - np.dot(v, n) * n

In [4]:
#%output backend="plotly"
def dynamic_name(group, name):
    """
    Generates a standardized dynamic trace name.

    Args:
        group (str): Type of the trace ("plane", "vector", "arc", etc.)
        name (str): Descriptive name of the trace.

    Returns:
        str: Combined dynamic trace name.
    """
    return f"dynamic:{group}:{name}"
def replace_traces(fig, new_traces, group="dynamic"):
    """
    Safely replaces all traces in the figure that belong to the given dynamic group.
    """
    prefix   = group if group.endswith(":") else f"{group}:"
    fig.data = [trace for trace in fig.data if not (hasattr(trace, "name") and trace.name.startswith("dynamic:"))]
    fig.add_traces(new_traces)
def create_plane(origin, v1, v2, size=2.0):
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    points = [origin + a * v1 + b * v2 for a in [-size, size] for b in [-size, size]]
    x = np.array([p[0] for p in points]).reshape(2, 2)
    y = np.array([p[1] for p in points]).reshape(2, 2)
    z = np.array([p[2] for p in points]).reshape(2, 2)
    return x, y, z
def plot_vector_pair(u_vec, v_vec, scale=1.6, u_color='darkred', v_color='blue', name_prefix='principal'):
    return [
        go.Scatter3d(x=[0, u_vec[0]*scale], y=[0, u_vec[1]*scale], z=[0, u_vec[2]*scale],
                     mode='lines', line=dict(color=u_color, width=6),
                     name=dynamic_name("vector", f"{name_prefix} u"),
                     showlegend=False),
        go.Scatter3d(x=[0, v_vec[0]*scale], y=[0, v_vec[1]*scale], z=[0, v_vec[2]*scale],
                     mode='lines', line=dict(color=v_color, width=6),
                     name=dynamic_name("vector", f"{name_prefix} v"),
                     showlegend=False)
    ]
def angle_arc(u, v, origin=None, color='black', name_prefix="arc"):
    if origin is None:
        origin = np.zeros(3)

    # Generate points along the arc
    u_norm = u / np.linalg.norm(u)
    v_norm = v / np.linalg.norm(v)
    angle = np.arccos(np.clip(np.dot(u_norm, v_norm), -1, 1))

    if np.isclose(angle, 0) or np.isclose(angle, np.pi):
        return None  # No arc needed for 0° or 180°

    axis = np.cross(u_norm, v_norm)
    if np.linalg.norm(axis) < 1e-6:
        return None  # Axis is too small

    axis /= np.linalg.norm(axis)
    n_points = 30
    angles = np.linspace(0, angle, n_points)
    arc_points = []

    for θ in angles:
        rot = rotation_matrix(axis, θ)
        p = rot @ u_norm
        arc_points.append(p + origin)

    arc_points = np.array(arc_points)

    return go.Scatter3d(
        x=arc_points[:, 0],
        y=arc_points[:, 1],
        z=arc_points[:, 2],
        mode='lines',
        line=dict(color=color, width=4, dash='dash'),
        showlegend=False,
        name=dynamic_name("arc", name_prefix)
    )
def plot_intersection_line(u1, u2, v1, v2, scale=2.5, name_prefix="Intersection Line"):
    nU = np.cross(u1, u2)
    nV = np.cross(v1, v2)
    line_dir = np.cross(nU, nV)

    if np.linalg.norm(line_dir) < 1e-6:
        return None  # No well-defined intersection

    line_dir /= np.linalg.norm(line_dir)

    return go.Scatter3d(
        x=[-scale*line_dir[0], scale*line_dir[0]],
        y=[-scale*line_dir[1], scale*line_dir[1]],
        z=[-scale*line_dir[2], scale*line_dir[2]],
        mode='lines',
        line=dict(color='black', width=3, dash='dash'),
        name=dynamic_name("intersection", name_prefix),
        showlegend=False  # Hide from legend to avoid jitter
    )

In [9]:
class PrincipalAnglePlayer(pn.viewable.Viewer):
    offset  = param.Number(default=0.0,  bounds=(-5.0, 5.0), label="Z Offset for Plane V")
    theta_1 = param.Integer(default=0,  bounds=(-1800, 1800), label="theta_1")  # 10*degrees
    theta_2 = param.Integer(default=90, bounds=(-1800, 1800), label="theta_2")  # 10*degrees

    r     = param.Number(default=3.5, bounds=(1.0, 10.0), label="Camera Distance")
    theta = param.Number(default=45,  bounds=(0, 360),    label="Azimuth (θ°)")
    phi   = param.Number(default=30,  bounds=(-90, 90),   label="Elevation (φ°)")

    def __init__(self, **params):
        super().__init__(**params)
        self.fig_pane = pn.pane.Plotly(height=650, config={'responsive': True})
        self.distances_pane = pn.pane.Markdown("", sizing_mode="stretch_width")

        self._locked_camera = None
        self.lock_button = pn.widgets.Button(name="Lock Current Camera", button_type="primary", width=200)
        self.lock_button.on_click(self._lock_camera_from_plot)

        self.param.watch(self._update_data_only, ['theta_1', 'theta_2', 'offset'])
        self.param.watch(self._update_camera_only, ['r', 'theta', 'phi'])

        self._current_camera = self._camera_eye()
        self._update_data_only()

    def _svd_from_planes(self, u1, u2, v1, v2):
        U, _ = np.linalg.qr(np.stack([u1, u2], axis=1))
        V, _ = np.linalg.qr(np.stack([v1, v2], axis=1))
        return np.linalg.svd(U.T @ V), U, V

    def compute_principal_vectors_and_angles(self, u1, u2, v1, v2):
        (W1, s, W2T), U, V = self._svd_from_planes(u1, u2, v1, v2)
        principal_U = U @ W1
        principal_V = V @ W2T.T
        angles = np.arccos(np.clip(s, -1, 1))
        return principal_U, principal_V, np.degrees(angles)

    def _camera_eye(self):
        θ, φ = np.radians(0.1*self.theta), np.radians(0.1*self.phi)
        zoom_factor = self.r
        x = zoom_factor * np.cos(θ) * np.cos(φ)
        y = zoom_factor * np.sin(θ) * np.cos(φ)
        z = zoom_factor * np.sin(φ)
        return dict(x=x, y=y, z=z)

    def _lock_camera_from_plot(self, _=None):
        if self.fig_pane.object and hasattr(self.fig_pane.object.layout.scene, "camera"):
            cam = self.fig_pane.object.layout.scene.camera.eye
            self._locked_camera = dict(x=cam.x, y=cam.y, z=cam.z)

    def _update_camera_only(self, *_):
        if self._locked_camera:
            self._current_camera = self._locked_camera
        else:
            self._current_camera = self._camera_eye()

        if self.fig_pane.object:
            self.fig_pane.object.update_layout(scene_camera=dict(eye=self._current_camera))

    def _update_data_only(self, *_):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])
        origin = np.zeros(3)

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        xU, yU, zU = create_plane(origin, u1, u2)
        xV, yV, zV = create_plane(origin + np.array([0, 0, self.offset]), v1, v2)

        principal_U, principal_V, angles = self.compute_principal_vectors_and_angles(u1, u2, v1, v2)

        # --- Create all new traces ---
        traces = [
            go.Surface(x=xU, y=yU, z=zU, opacity=0.85, showscale=False,
                       colorscale=[[0, '#989F7A'], [1, '#989F7A']],
                       name=dynamic_name("plane", "Plane U"), showlegend=False),
            go.Surface(x=xV, y=yV, z=zV, opacity=0.65, showscale=False,
                       colorscale=[[0, '#bbbbbb'], [1, '#bbbbbb']],
                       name=dynamic_name("plane", "Plane V"), showlegend=False)
        ]

        traces += plot_vector_pair(principal_U[:, 0], principal_V[:, 0], name_prefix='p1')
        traces += plot_vector_pair(principal_U[:, 1], principal_V[:, 1], name_prefix='p2')

        arc1 = angle_arc(principal_U[:, 0], principal_V[:, 0], origin=origin, color='black', name_prefix="θ₁ arc")
        arc2 = angle_arc(principal_U[:, 1], principal_V[:, 1], origin=origin, color='black', name_prefix="θ₂ arc")
        if arc1: traces.append(arc1)
        if arc2: traces.append(arc2)

        inter_line = plot_intersection_line(u1, u2, v1, v2)
        if inter_line: traces.append(inter_line)

        title_text = f"Principal Angles: θ₁ = {angles[0]:.2f}°, θ₂ = {angles[1]:.2f}°"
        # --- Full new figure (flicker-free), reusing locked or current camera ---
        camera_settings = dict(
            eye=self._locked_camera if self._locked_camera else self._camera_eye(),
            projection=dict(type="orthographic")
        )

        fig = go.Figure(data=traces)
        fig.update_layout(
            title=title_text,
            scene=dict(
                aspectmode='cube',
                xaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='X'),
                yaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='Y'),
                zaxis=dict(tickmode='array', tickvals=[-1, 0, 1, 2, 3], range=[-1.5, 3.0], title='Z'),
                camera=camera_settings
            ),
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(size=12)),
            margin=dict(l=10, r=10, t=40, b=10),
            showlegend=True
        )

        self.fig_pane.object = fig
        self.distances_pane.object = self._compute_distances_markdown()

    def _compute_distances_markdown(self):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        (_, s, _), _, _ = self._svd_from_planes(u1, u2, v1, v2)
        s = np.clip(s, 0, 1)
        angles = np.arccos(s)

        spectral   = np.sin(np.max(angles))
        chordal    = np.sqrt(np.sum(np.sin(angles)**2))
        projection = np.sqrt(2) * chordal
        geodesic   = np.sqrt(np.sum(angles**2))

        text = f"""
        **Distances:**
        - **Spectral Distance:**   {spectral:20.4f}
        - **Chordal Distance:**    {chordal:20.4f}
        - **Projection Distance:** {projection:20.4f}
        - **Geodesic Distance:**   {geodesic:20.4f} radians
        """
        return text

    def __panel__(self):
        return pn.Row(
            pn.Column(
                pn.pane.Markdown("## Principal Angle Visualizer"),
                self.fig_pane,
                width=500,
            ),
            pn.Column(
                r"## Controls: θ₁ (Intersection), θ₂ (Inclination).",
                self.distances_pane,
                pn.Spacer(height=20),
                self.lock_button,
                pn.Spacer(height=20),

                "theta_1", pn.widgets.Player.from_param(self.param.theta_1, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                "theta_2", pn.widgets.Player.from_param(self.param.theta_2, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                width=500,
            ),
        )

# 
<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Metrics Induced by Principal Angles</strong><br>
</div>

# 1. Review: Principal Angles Between Subspaces

In the [**previous notebook**](HyperplanePrincipalAngles.ipynb), we introduced **principal angles** as a way to quantify the alignment between two subspaces of $\mathbb{R}^n$.

Given two subspaces
$\; \mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n,\;$
with dimensions $\;\dim(\mathcal{U}) = k$ and $\dim(\mathcal{V}) = \ell,\;$<br>
the principal angles
$\; 0^\circ \leq \theta_1 \leq \theta_2 \leq \cdots \leq \theta_d \leq 90^\circ,\;$
where $\;d = \min(k, \ell),\;$<br>
measure the smallest, second smallest, ..., up to the largest angle between the two subspaces.

<div style="background-color:#F2F5A9;color:black;">

**Definition:**
The principal angles $\theta_i$ between two subspaces $\mathcal{U}$ and $\mathcal{V}$ are defined recursively by

$\qquad \cos(\theta_i) = \max_{u \in \mathcal{U}, v \in \mathcal{V}} \quad u^\top v$

$\qquad$ subject to $\; \|u\| = \|v\| = 1, \qquad u^\top u_j = 0, \quad v^\top v_j = 0 \quad \text{for all } j < i,$<br>
$\qquad$ where $u_j, v_j$ are the previously chosen principal vectors.
</div>

**Computation:**

Given orthonormal bases $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$ for the subspaces,<br>
the **singular values of $\mathbf{U^\top V}$ provide the cosines of the principal angles** $\theta_i$

$\qquad U^\top V = W_1\ \Sigma\ W_2^\top,$

where the singular values $\sigma_1, \sigma_2, \dots, \sigma_d$ satisfy

$\qquad \sigma_i = \cos(\theta_i).$

Thus:
- Small singular values correspond to large principal angles (subspaces are more orthogonal).
- Large singular values correspond to small principal angles (subspaces are more aligned).

____

**Quick Intuition:**<br>
- If $\theta_1 = 0^\circ$, the subspaces share a common direction exactly.
- If $\theta_d = 90^\circ$, there is a direction in $\mathcal{U}$ orthogonal to $\mathcal{V}$.

Principal angles thus provide a full spectrum of how two subspaces align — not just "are they close" but *how close in each dimension*.

____

In this notebook, we build on this foundation to show:

$\qquad$ how principal angles naturally define **distance metrics** between subspaces, and<br>
$\qquad$ how different metrics emphasize different aspects of subspace similarity or separation.

# 2. Principal Angles Induce Distance Metrics

Principal angles not only describe the geometric relationship between two subspaces,<br>
but also naturally induce various [**distance metrics**](23_MetricSpaces) between them.

Each metric emphasizes different aspects of the subspace alignment:
- Some focus on the worst-case deviation,
- Others measure an overall energy of misalignment,
- Some capture the full geometric "path length" between the subspaces.

____

We now define the most common distance metrics derived from principal angles.

## 2.1 Spectral Distance

<div style="display:flex;">
<div style="float:left;padding-right:1cm;padding-top:0.5cm;background-color:#F2F5A9;">

**Definition:** The **spectral distance** measures the maximum misalignment<br>$\qquad$ between $\mathcal{U}$ and $\mathcal{V}$:

$\qquad d_{\text{spec}}(\mathcal{U}, \mathcal{V}) = \sin(\theta_{\max})$

where $\theta_{\max}$ is the largest principal angle.

- Sensitive only to the worst-case deviation.
- Small if and only if all principal angles are small.

**Remark:** This is **not a true metric (it fails the triangle Inequality)**
</div>
<div style="float:left;padding-left:1cm;border-left:2px solid black;">

**Example:**

Consider two subspaces in $\mathbb{R}^3$:

- $\mathcal{U}$ is the xy-plane.
- $\mathcal{V}$ is a plane rotated by $10^\circ$ along the x-axis, but by $80^\circ$ along the y-axis.

Thus, the principal angles are approximately $\; \theta_1 = 10^\circ, \; \theta_2 = 80^\circ.$

$\qquad d_{\text{spec}}(\mathcal{U}, \mathcal{V}) = \sin(80^\circ) \approx 0.9848.$

Even though one direction is very close,<br>
**the large $80^\circ$ misalignment dominates the spectral distance.**
</div></div>

## 2.2 Chordal Distance

<div style="display:flex;">
<div style="float:left;padding-right:1cm;padding-top:0.5cm;padding-bottom:0.5cm;background-color:#F2F5A9;">

**Definition:** The **chordal distance** captures the cumulative misalignment<br>$\qquad$ across all principal angles:

$\qquad d_{\text{chordal}}(\mathcal{U}, \mathcal{V}) = \left( \sum_{i=1}^d \sin^2(\theta_i) \right)^{1/2}$

- **Measures the total "energy" of misalignment.**
- More robust to isolated large angles than spectral distance.
- Treats all principal angles equally.

**Remark:** This is a **true metric** (it satisfies all metric properties).

</div>
<div style="float:left;padding-left:1cm;border-left:2px solid black;">

**Example:**

Consider the same subspaces in $\mathbb{R}^3$ as before:
- $\mathcal{U}$ is the xy-plane.
- $\mathcal{V}$ is a plane rotated by $10^\circ$ along the x-axis, and $80^\circ$ along the y-axis.

Thus, the principal angles are approximately: $\;\theta_1 = 10^\circ, \; \theta_2 = 80^\circ.$

$\qquad\begin{aligned}
d_{\text{chordal}}(\mathcal{U}, \mathcal{V}) &= \left( \sin^2(10^\circ) + \sin^2(80^\circ) \right)^{1/2} \\
&\approx 0.9999.
\end{aligned}$

Notice that although one principal angle ($10^\circ$) is small,<br>
the large $80^\circ$ angle dominates the chordal distance after accumulation.

</div></div>

# 2.3 Projection Distance

<div style="display:flex;"><div style="float:left;padding-right:1cm;padding-top:0.5cm;background-color:#F2F5A9;">

**Definition:** The **projection distance** measures the difference between<br>
$\qquad$ the orthogonal projectors onto $\mathcal{U}$ and $\mathcal{V}$:

$\qquad d_{\text{proj}}(\mathcal{U}, \mathcal{V}) = \| P_{\mathcal{U}} - P_{\mathcal{V}} \|_F$

where

$\qquad P_{\mathcal{U}} = UU^\top, \qquad P_{\mathcal{V}} = VV^\top.$

- Captures how much the two projections differ globally.
- Equivalent to chordal distance up to a constant scaling factor.
- Directly tied to matrix approximation and stability.

**Remark:** This is a **true metric** (it satisfies all metric properties).

</div>
<div style="float:left;padding-left:1cm;border-left:2px solid black;">

**Example:**

Consider the same subspaces in $\mathbb{R}^3$ as before:

- $\mathcal{U}$ is the xy-plane.
- $\mathcal{V}$ is a plane rotated by $10^\circ$ along the x-axis and $80^\circ$ along the y-axis.

Thus, the principal angles are approximately $\;\theta_1 = 10^\circ, \; \theta_2 = 80^\circ.$

$\begin{aligned}
d_{\text{proj}}(\mathcal{U}, \mathcal{V}) &= \sqrt{2} \, d_{\text{chordal}}(\mathcal{U}, \mathcal{V}) \\
&= \sqrt{2} \times \left( \sin^2(10^\circ) + \sin^2(80^\circ) \right)^{1/2} \\
&\approx 1.4142.
\end{aligned}$

Here, the projection distance scales the chordal distance by a factor of $\sqrt{2}$.<br>
It still reflects the significant misalignment dominated by the $80^\circ$ principal angle.

</div></div>

## 2.4 Geodesic Distance

<div style="display:flex;"><div style="float:left;padding-right:1cm;padding-top:0.5cm;padding-bottom:1cm; background-color:#F2F5A9;">

**Definition:** The **geodesic distance** measures the true "shortest path"<br>
$\qquad$ between $\mathcal{U}$ and $\mathcal{V}$ on the [Grassmannian manifold](GrassmannianManifold.ipynb)

$\qquad d_{\text{geo}}(\mathcal{U}, \mathcal{V}) = \left( \sum_{i=1}^d \theta_i^2 \right)^{1/2}$

- Uses the principal angles themselves (not their sine or cosine).
- Treats the subspaces as points on a curved manifold (Grassmannian).
- True Riemannian distance between subspaces.

**Remark:** This is a **true metric** (it satisfies all metric properties).

</div>
<div style="float:left;padding-left:1cm;border-left:2px solid black;">

**Example:**

Consider the same subspaces in $\mathbb{R}^3$ as before:

- $\mathcal{U}$ is the xy-plane.
- $\mathcal{V}$ is a plane rotated by $10^\circ$ along the x-axis and $80^\circ$ along the y-axis.

Thus, the principal angles are approximately:$\; \theta_1 = 10^\circ \approx 0.1745 \text{ rad}, \; \theta_2 = 80^\circ \approx 1.3963 \text{ rad}.$

$\begin{aligned}
d_{\text{geo}}(\mathcal{U}, \mathcal{V}) &= \left( (0.1745)^2 + (1.3963)^2 \right)^{1/2} \\
&\approx 1.4073 \text{ rad}.
\end{aligned}$

Unlike the spectral, chordal, or projection distances,<br>
the geodesic distance **directly accumulates** the principal angles themselves,<br>
not just their sine or cosine.

</div></div>

## 2.5 Summary

In [10]:
PrincipalAnglePlayer().servable()

Row
    [0] Column(width=500)
        [0] Markdown(str)
        [1] Plotly(Figure, config={'responsive': True}, height=650)
    [1] Column(width=500)
        [0] Markdown(str)
        [1] Markdown(str, sizing_mode='stretch_width')
        [2] Spacer(height=20)
        [3] Button(button_type='primary', name='Lock Current Camera', width=200)
        [4] Spacer(height=20)
        [5] Markdown(str)
        [6] Player(end=1800, interval=10, loop_policy='reflect', name='theta_1', start=-1800, step=5, width=250)
        [7] Markdown(str)
        [8] Player(end=1800, interval=10, loop_policy='reflect', name='theta_2', start=-1800, step=5, value=90, value_throttled=90, width=250)

<div style="float:left; padding-left:1cm;">

| Metric | Formula | Emphasis | True Metric | Name Origin |
|:---|:---|:---|:---|:---|
| Spectral | $\sin(\theta_{\max})$ | Worst-case misalignment | **No** (fails triangle inequality) | Depends on largest singular value (spectrum) |
| Chordal | $\left( \sum \sin^2(\theta_i) \right)^{1/2}$ | Total misalignment energy | Yes | Straight-line (chord) distance between subspaces |
| Projection | $\| P_{\mathcal{U}} - P_{\mathcal{V}} \|_F$ | Frobenius norm of projectors | Yes | Distance between orthogonal projectors |
| Geodesic | $\left( \sum \theta_i^2 \right)^{1/2}$ | Geometric path length | Yes | Shortest curved path (geodesic) on Grassmannian |

<br>

**Remark:**<br>
$\qquad$ The projection distance and the chordal distance are related by a constant factor:

$\qquad\qquad d_{\text{proj}}(\mathcal{U}, \mathcal{V}) = \sqrt{2}\; \, d_{\text{chordal}}(\mathcal{U}, \mathcal{V}).$

$\qquad$ Thus, although they emphasize similar behaviors, the projection distance consistently scales larger by $\sqrt{2}$ compared to the chordal distance.
</div>

# 3. Small Examples and Visual Comparisons

In this section, we work through small concrete examples to develop intuition about the distance metrics derived from principal angles.

We will:

- Compute all four distances (spectral, chordal, projection, geodesic) for a simple example.
- Compare how the distances behave relative to each other.
- Prepare for larger dynamic experiments in later sections.

## 3.1 Static Example: Two 2D Subspaces in $\mathbb{R}^3$

We now work through a concrete example step-by-step.

#### **Step 1:** Define **orthonormal bases** for $\mathcal{U}$ and $\mathcal{V}$
$\qquad$ When necessary, use the **Gram Schmidt Procedure** to ensure orthogonality and unit length

In [ ]:
%%julia
U = [1 0; 0 1; 0 0]
Ṽ = [cosd(10) 0; sind(10) cosd(80); 0 sind(80)]
V = Matrix(qr(Ṽ).Q)
py_show( L"U = ", U, L", \quad V =", (V, number_formatter=x->round_value(x,3)) )

In [ ]:
%%julia
rref([V Ṽ ])

#### **Step 2:** Compute matrix $U^T V$ of inner products

In [ ]:
%%julia
M = U'V
py_show( L"M = U^T V =", (M, number_formatter=x->round_value(x,3)) )

#### **Step 3:** compute an SVD of $U^T V$
$\qquad$ Note we only need a reduced SVD

In [ ]:
%%julia
S = svd(M)
A, Σ, B = S.U, S.S, S.V
py_show( "SVD(M) = ", set(A, Diagonal(Σ), B, number_formatter=x->round_value(x,3), setstyle=:array, separator=""))

#### Step 4: Compute Principal Angles and (optionally) Principal Angles
$\qquad$ Clamp the $\sigma$ values to [0,1] to avoid numerical problems

In [ ]:
%%julia
θ = acosd.(clamp.(Σ, 0, 1))
py_show( "Principal angles in Degrees: ", (x->round(x, digits=3)).(θ'))

#### Step 5: Compute the Principal Vectors

In [ ]:
%%julia
u₁ = U * A[:, 1]
u₂ = U * A[:, 2]

v₁ = V * B[:, 1]
v₂ = V * B[:, 2]

display( py_show( "Principal Vector pair for angle ", L"\; \theta_1 \approx ", θ[1], L",\quad \; u_1 \approx ", u₁, L", \; v_1 \approx ", v₁, number_formatter=x->round_value(x,3)))
display( py_show( "Principal Vector pair for angle ", L"\; \theta_2 \approx ", θ[2], L", \; u_2 \approx ", u₂, L", \; v_2 \approx ", v₂, number_formatter=x->round_value(x,3)))

____
**Remark:**
Even after QR orthonormalization, the computed principal angles deviate slightly from the intended $10^\circ$ and $80^\circ$ values.<br>
$\qquad$ **We see $0^\circ$ and $80.149^\circ$.**

This discrepancy arises because the original vectors defining $V$ were not exactly orthogonal and not fully independent:<br>
$\qquad$ their spans were already close to being linearly dependent!
As a result, the QR result will not quite satisfy $\mathscr{C}(V) = \mathscr{C}(Q)$<br>
$\qquad$ but instead yield a second vector that is slightly rotated compared to the exact result.<br>
$\qquad$ $\therefore$ Choose bases with care!

This behavior can be diagnosed by computing a QR decomposition of $\begin{pmatrix} V_{\text{original}} & V_{\text{orthonormalized}} \end{pmatrix}$,<br>
$\qquad$ which reveals a third direction representing the adjustment made during orthonormalization.

In [ ]:
%%julia
vecs = Matrix(qr([ V Ṽ ]).Q)
py_show( L"Q \left( \begin{pmatrix} V_{\text{original}} & V_{\text{orthonormalized}} \end{pmatrix} \right) = ", vecs, L"\quad", "has 3 vectors, not 2! ", number_formatter=x->round_value(x,3))

# 3.2 Dynamic Distance Metrics: Interactively Vary First Angle

In [7]:
hv.extension('bokeh', logo=False)

def compute_distances(theta1_deg):
    theta2_list = np.linspace(0, 90, 100)  # Sweep second angle from 0 to 90 degrees

    spectral_distances   = []
    chordal_distances    = []
    projection_distances = []
    geodesic_distances   = []

    U = np.eye(3)[:, :2]

    for theta2_deg in theta2_list:
        theta1 = np.radians(theta1_deg)
        theta2 = np.radians(theta2_deg)

        # Build V
        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])

        v1      = np.array([np.cos(theta1), np.sin(theta1), 0])
        v1_perp = np.array([-np.sin(theta1), np.cos(theta1), 0])
        v2 = np.cos(theta2) * v1_perp + np.sin(theta2) * np.array([0, 0, 1])

        V = np.stack([v1, v2], axis=1)
        V, _ = np.linalg.qr(V)

        # Principal angles
        M = U.T @ V
        _, s, _ = np.linalg.svd(M)
        s = np.clip(s, 0, 1)
        angles = np.arccos(s)

        # Distances
        spectral   = np.sin(np.max(angles))
        chordal    = np.sqrt(np.sum(np.sin(angles)**2))
        projection = np.sqrt(2) * chordal
        geodesic   = np.sqrt(np.sum(angles**2))

        spectral_distances.append(    spectral  )
        chordal_distances.append(     chordal   )
        projection_distances.append(  projection)
        geodesic_distances.append(    geodesic  )

    overlay = hv.Overlay([
        hv.Curve((theta2_list, spectral_distances),   label="Spectral"),
        hv.Curve((theta2_list, chordal_distances),    label="Chordal"),
        hv.Curve((theta2_list, projection_distances), label="Projection"),
        hv.Curve((theta2_list, geodesic_distances),   label="Geodesic"),
    ]).opts(
        width=600, height=400,
        xlabel="Second Principal Angle (θ₂) [degrees]",
        ylabel="Distance",
        title=f"Distances vs θ₂ (Fixed θ₁ = {theta1_deg}°)",
        legend_position="top_left",
        show_grid=True
    )

    return overlay

# --------------------------------------------------------------------------
pn.interact(compute_distances, theta1_deg=(0, 90, 1))

Column(sizing_mode='fixed')
    [0] Column
        [0] IntSlider(end=90, name='theta1_deg', value=45)
    [1] Row(sizing_mode='fixed')
        [0] HoloViews(Overlay, height=400, name='interactive00262', sizing_mode='fixed', width=600)

Use the slider above to fix the first principal angle ($\theta_1$),
and observe how each distance metric varies as the second principal angle ($\theta_2$) sweeps from $0^\circ$ to $90^\circ$.

Each curve corresponds to a different distance definition:
- **Spectral Distance** (worst-case deviation),
- **Chordal Distance** (total energy),
- **Projection Distance** (norm of projector difference),
- **Geodesic Distance** (true manifold path length).

# 4. Take Away